# Исследовательский проект по НИС «Анализ данных в Python»
<div style="text-align: right">
    Проект подготовлен студентами БПИ228
    <br>
    Лысиным Кириллом и Гакал Анжеликой
</div>

## Часть 2. Сбор данных из интернета и составление датасета.
Данную часть задания мы выполняли первой, так как её результаты являются предметом аналаза в следующей части задания.

После обсуждения вариантов мы решили собирать информацию о товарах и услугах, размещённых на сервисе объявлений Юла.
Выбором данного сервиса объявлений обусловлен ряд решений в области реализации:
<br>
<p style="margin-left: 50px;">
    <ul>
        <li>Библиотека requests, пройденная на занятиях, не позволяет получить все необходимые данные со страницы, так как часть контента отображается только после выполнения js-скриптов, что не происходит при использовании вышеуказанной библиотеки, поэтому для получения данных с сайта исопльзуется библиотека selenium.</li>
        <li>Работа с динамически генерируемым содержимым сайта делает невозможной (или крайне затруднительной) унификацию процесса, так как для разных устройств это содержимое может отличаться. Этим фактом обусловлена заточенность прогаммы, осуществляющей парсинг, под конкретное устройство, но изменения, необходимые для работы программы на другом устройстве могут быть внесены за незначительный промежуток времени.</li>
    </ul>
</p>

In [15]:
import datetime
from selenium import webdriver
import pandas as pd
import time
from bs4 import BeautifulSoup
import os.path


    
driver = webdriver.Chrome()

with open('visited_links.txt', 'w+') as f:
    visited_links = set(f.readlines())


def get_item_info(url):
    
    info_dict = {}
    soup = get_soup(url)
    if(soup == None):
        return
    
    general_info = str(soup.findAll('meta', {'property':"og:description"})[0])

    # Сохранение данных о товаре/услуге в словарь
    info_dict['Name'] = get_item_name(general_info)
    info_dict['Price (in rubles)'] = get_item_price(general_info)
    info_dict['Category'] = get_item_category(general_info)
    info_dict['Region'] = get_item_region(soup)
    info_dict['Time/date of placement'] = get_item_placement_time(soup)
    info_dict['Added to favourites'] = get_favourites(soup)
    info_dict['Number of seller\'s ads'] = get_number_of_ads(soup)
    info_dict['Seller rating'] = get_seller_rating(soup)
    info_dict['Number of views'] = get_number_of_views(soup)
    info_dict['Number of images'] = get_number_of_images(soup)
    info_dict['Link'] = url
    
    return info_dict

def get_soup(url, counter = 0):

    time.sleep(1)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    # Если не удалось сразу получить страницу, то программа пробует еще несколько раз
    except:
        if(counter < 3):
            return get_soup(url, counter + 1)
    # Если после несольких попыток так и не удалось считать содержимое страницы, то этот метод вернет None

def get_item_name(info):
    try:
        name = info[15:info.find('–') - 1]
        return name
    except:
        return None

def get_item_price(info):
    if not('Цена: ' in info):
        return 0
    price = float(info[info.find('Цена: ') + 6:info.find('руб.') - 1].replace(' ', ''))
    return price

def get_item_category(info):
    try:
        category = info[info.find('раздела «') + 9:info.find('».')]
        return category
    except:
        return None

def get_item_region(soup):
    try:
        region = list(soup.findAll('span', class_='sc-cOxWqc hNYaaO'))[0].text
        return region
    except:
        return None

def get_item_placement_time(soup):
    try:
        s = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))[-1].text
        now = datetime.datetime.now()
        if('Сегодня в' in s):
            return datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:]))
        if('Вчера в' in s):
            return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-1))
        if('Позавчера в' in s):
            return (datetime.datetime(now.year, now.month, now.day, int(s[-5:-3]), int(s[-2:])) + datetime.timedelta(days=-2))


        months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
        for j in range(len(months)):
            if(months[j].lower() in s):
                return datetime.datetime(now.year, j + 1, int(s[:2]), 12)
        return datetime.datetime(day=int(s[:2]), month=int(s[3:5]), year=int(s[6:10]), hour = 12)
    except:
        return None


def get_favourites(soup):
    try:
        favourites_count = list(soup.findAll('dd', class_="sc-cOxWqc sc-fVmuvm eOPaPs dVayGV"))
        return int(favourites_count[-3].text)
    except:
        return None

def get_number_of_ads(soup):
    try:
        n = list(soup.findAll('span', {'data-test-component':"UserNameClick"}))[0].text
        n = int(n[n.find('(')+1:n.find(' объявл')])
        return n
    except:
        return None

def get_seller_rating(soup):
    try:
        rating = soup.find('span', class_="sc-cOxWqc eDoIYl")
        return float(rating.text.replace(',', '.')) if rating != None else None
    except:
        return None

def get_number_of_views(soup):
    try:
        number_of_views = list(soup.findAll('dd', class_='sc-cOxWqc sc-fVmuvm eOPaPs dVayGV'))
        return int(number_of_views[-2].text)
    except:
        return None

def get_number_of_images(soup):
    try:
        images = list(soup.findAll('img', class_="sc-fBnnfK"))
        return len(images)
    except:
        return None

def get_links(scroll_count):
    
    url = r'https://youla.ru/'
    driver.get(url)
    time.sleep(1)
    
    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    links = [str(i.a) for i in soup.findAll('span', class_="sc-llGDqb sc-gqgnwQ fEAASo hZGRky")]
    links = [r'https://youla.ru' + s[s.find(r'href="') + 6 : s.find(r'" rel')] for s in links]
    return links



n = 100
for i in range(n):
    items = []
    print(f"Processing batch {i + 1} out of {n}:", end='')
    links = get_links(1)
    for i in range(len(links)):
        if(int(i * 100/len(links))//5 > int((i - 1) * 100/len(links))//5):
            print(' .', end='')
        link = links[i]
        if(link in visited_links):
            continue
        items.append(get_item_info(link))
        visited_links.add(link)
    print(' ✓')

    df = pd.DataFrame(items)
    if(len(df) > 0):
        df = df.set_index('Name')
        if(os.path.isfile('items.xlsx')):
            prev_df = pd.read_excel('items.xlsx', index_col = 0)
            df = pd.concat([prev_df, df])

        df.to_excel('items.xlsx')

print("Done! The data has been saved to the file items.xlsx")

with open('visited_links.txt', 'a') as f:
    for line in visited_links:
        f.write(f"{line}\n")



driver.quit()
df

Processing batch 1 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 2 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 3 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 4 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 5 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 6 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 7 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 8 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 9 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 10 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 11 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 12 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 13 out of 100: . . . . . . . . . . . . . . . . . . . . ✓
Processing batch 14 out of 100: . . . . . . . .

,Price (in rubles),Category,Region,Time/date of placement,Added to favourites,Number of seller's ads,Seller rating,Number of views,Number of images,Link
Name,,,,,,,,,,
Брюки 80-92,0.0,Детский гардероб,Москва,2023-01-31 22:00:00,2.0,40.0,5.0,14.0,5,https://youla.ru/moskva/detskaya-odezhda/shtan...
Туфли размер 30. Бесплатно,0.0,Детский гардероб,Химки,2023-01-31 21:59:00,0.0,54.0,5.0,18.0,0,https://youla.ru/himki/detskaya-odezhda/obuv/t...
Кошка бесплатно,0.0,Животные,Москва,2023-01-31 21:59:00,7.0,1.0,NaN,307.0,3,https://youla.ru/all/zhivotnye/koshki/bielaia-...
Demix на 6 лет на синтепоне,0.0,Детский гардероб,Котельники,2023-01-31 21:59:00,0.0,111.0,5.0,127.0,4,https://youla.ru/kotelniki/detskaya-odezhda/ve...
Красивый пёс в добрые руки,0.0,Животные,Люберцы,2023-01-31 21:59:00,151.0,19.0,NaN,8402.0,10,https://youla.ru/lyubertsy/zhivotnye/sobaki/kr...
...,...,...,...,...,...,...,...,...,...,...
Игровая приставка Nintendo Switch,17499.0,Хобби и развлечения,Москва,2023-01-31 23:02:00,0.0,2.0,4.8,7.0,0,https://youla.ru/moskva/hobbi-razvlecheniya/ko...
Фотоаппарат Nikon d3300,15000.0,Фото- и видеокамеры,Москва,2023-01-05 12:00:00,3.0,1.0,NaN,42.0,10,https://youla.ru/moskva/foto-video/fotoapparat...
Шотландские золотые и колорные котята,12000.0,Животные,Владимир,2023-01-27 12:00:00,5.0,NaN,NaN,219.0,10,https://youla.ru/vladimir/zhivotnye/koshki/sho...


## Часть 1. Анализ датасета.